# Models, Prompt, and Output Parsers

We will practise how to create a LLM with Llama-2-7b-chat-finetune and HuggingFace pipeline. We will take reference from the LangChainForLLMApplication course of DeepLearing.ai.

## Setup Environment

In [1]:
# %%capture
# !pip install langchain
# !pip install langchain-hub
# !pip install langchain-community langchain-huggingface
# !pip install huggingface_hub transformers
# !pip install sentence_transformers==2.2.2
# !pip install chromadb faiss accelerate
# !pip install -U bitsandbytes
# !pip install tiktoken python-dotenv

## Import Modules

In [2]:
# llm modules
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from transformers import GenerationConfig
from langchain.llms import HuggingFacePipeline

# prompt modules
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

# output parser
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

# base modules
import os
import torch
import warnings

warnings.filterwarnings("ignore")

## Create LLM from Huggingface

We will create a LLM by using Llama2-7b-chat model from huggingface. OpenAI require API key and it is not free. Therefore, we will use our fine-tune Llama.

In [3]:
model_name = "minkhantycc/Llama-2-7b-chat-finetune-quantized"
device_map = {"": 0}

In [4]:
# bnb config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)


# base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model.generation_config = GenerationConfig(
    max_new_tokens = 256,
    temperature = 0.01,
    repetition_penalty = 1.15,
    do_sample = False,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.eos_token_id,
)

# pipeline
pipe = pipeline(
    task="text-generation",
    model=base_model,
    tokenizer=tokenizer,
    device_map=device_map,
    return_full_text=False
)

# llm
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def get_completion(prompt, model=llm):
    return model.invoke(prompt)

In [6]:
print(get_completion("<s>[INST] What is 1+1? [/INST]"))

 The answer to 1 + 1 is 2. 


## Prompt template

In [7]:
# create a Chat Prompt template
template_string = """<s>[INST]Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```[/INST]
"""

prompt_template = ChatPromptTemplate.from_template(template_string)
print(prompt_template)

input_variables=['style', 'text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='<s>[INST]Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```[/INST]\n'))]


In [8]:
# your style and email
custom_style = "American English \
in a calm and respectful tone"

custom_email = """
<s>[INST]Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey![/INST]
"""

# add them in the prompt template
customer_messages = prompt_template.format_messages(
                    style=custom_style,
                    text=custom_email)

print(customer_messages)
print(type(customer_messages))
print(type(customer_messages[0]))

[HumanMessage(content="<s>[INST]Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```\n<s>[INST]Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey![/INST]\n```[/INST]\n")]
<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [9]:
# get response from LLM
print(get_completion(customer_messages))

Oh no, it sounds like you had quite an unexpected mess on your hands there! It can be frustrating when things go awry like that, especially when they involve expensive electronics like blenders. But don't worry too much about the warranty not covering the cost of cleanup - at least you have some insurance against further damage or loss. Just take care of the situation as best you can, and try to stay level-headed through all the chaos. Good luck, matey!


## Output Parsers

In [10]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in 2 days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """<s>[INST]\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}[/INST]
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
ai_response = get_completion(messages)
print(ai_response)
print(type(ai_response))

{
"gift": true,
"delivery_days": -1,
"price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features."],
}

<class 'str'>


In [11]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a Python list.
}
```


In [12]:
review_template_2 = """<s>[INST]\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a Python list.

text: {text}

{format_instructions}[/INST]
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)
print(messages[0].content)

<s>[INST]For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in 2 days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```js

In [13]:
ai_response = get_completion(messages)
print(ai_response)

```json
{
	"gift": true,
	"delivery_days": "-1",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."],
}
```


In [14]:
output_dict = output_parser.parse(ai_response)
print(output_dict)
print(type(output_dict))

OutputParserException: Got invalid JSON object. Error: Expecting property name enclosed in double quotes: line 5 column 1 (char 180)